# ⛓ Chain 

In [ ]:
#| default_exp chains

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import asyncio, web3, os
from functools import wraps, reduce
from typing import List, TypeVar, Callable, Optional, Tuple, Dict
from fastcore.utils import patch
from web3 import AsyncWeb3, AsyncHTTPProvider, Account
from web3.eth.async_eth import AsyncContract
from sugar.config import ChainSettings, make_op_chain_settings, make_base_chain_settings
from sugar.helpers import normalize_address, MAX_UINT256, float_to_uint256, apply_slippage, get_future_timestamp
from sugar.abi import sugar, slipstream, price_oracle, router
from sugar.token import Token
from sugar.pool import LiquidityPool
from sugar.price import Price
from sugar.deposit import Deposit
from sugar.helpers import ADDRESS_ZERO, chunk, normalize_address

## Chain implementation 

In [ ]:
#| export

T = TypeVar('T')

def require_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    async def wrapper(self: 'Chain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return await f(self, *args, **kwargs)
    return wrapper

class Chain:
    account: Optional[Account]
    web3: AsyncWeb3
    sugar: AsyncContract
    router: AsyncContract

    def __init__(self, settings: ChainSettings):
        self.settings, self._in_context = settings, False

    @property
    def account(self) -> Account: return self.web3.eth.account.from_key(os.getenv("SUGAR_PK"))

    async def __aenter__(self):
        """Async context manager entry"""
        self._in_context = True
        self.web3 = AsyncWeb3(AsyncHTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=sugar)
        self.slipstream = self.web3.eth.contract(address=self.settings.slipstream_contract_addr, abi=slipstream)
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=price_oracle)
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=router)
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit"""
        self._in_context = False
        await self.web3.provider.disconnect()
        return None

### Get tokens

In [ ]:
#| export

@patch
@require_context
async def get_all_tokens(self: Chain, listed_only: bool = True) -> List[Token]:
    native = Token.make_native_token(self.settings.native_token_symbol, self.settings.wrapped_native_token_addr, self.settings.native_token_decimals)
    tokens = list(map(lambda t: Token.from_tuple(t), await self.sugar.functions.tokens(self.settings.pagination_limit, 0, ADDRESS_ZERO, []).call()))
    return [native] + (list(filter(lambda t: t.listed, tokens)) if listed_only else tokens)
   

### Get pools

In [ ]:
#| export

@patch
@require_context
async def get_pools(self: Chain) -> List[LiquidityPool]:
    pools, offset, limit = [], 0, self.settings.pool_page_size
    tokens = await self.get_all_tokens()
    prices = await self.get_prices(tokens)
    tokens, prices = {t.token_address: t for t in tokens}, {price.token.token_address: price for price in prices}

    while True:
        pools_batch = await self.sugar.functions.all(limit, offset).call()
        pools += pools_batch
        if len(pools_batch) < limit: break
        else: offset += limit

    return list(filter(lambda p: p is not None, map(lambda p: LiquidityPool.from_tuple(p, tokens, prices), pools)))

### Get prices

In [ ]:
#| export

# @cache_in_seconds(ORACLE_PRICES_CACHE_MINUTES * 60)
@patch
async def _get_prices(self: Chain, tokens: Tuple[Token]) -> List[Dict[str, int]]:
    # token_address => normalized rate
    result = {}
    rates = await self.prices.functions.getManyRatesToEthWithCustomConnectors(
        list(map(lambda t: t.wrapped_token_address or t.token_address, tokens)),
        False, # use wrappers
        self.settings.connector_tokens_addrs,
        10 # threshold_filer
    ).call()

    # rates are returned multiplied by eth decimals + the difference in decimals to eth
    # we want them all normalized to 18 decimals
    for cnt, rate in enumerate(rates):
        t, eth_decimals = tokens[cnt], self.settings.native_token_decimals
        if t.decimals == eth_decimals: nr = rate
        elif t.decimals < eth_decimals: nr = rate // (10 ** (eth_decimals - t.decimals))
        else: nr = rate * (10 ** (t.decimals - eth_decimals))
        result[t.token_address] = nr

    return result


@patch
@require_context
async def get_prices(self: Chain, tokens: List[Token]) -> List[Price]:
    """Get prices for tokens in target stable token"""

    eth_decimals = self.settings.native_token_decimals

    batches = await asyncio.gather(
        *map(
            # XX: lists are not cacheable, convert them to tuples so lru cache is happy
            lambda ts: self._get_prices(tuple(ts)),
            list(chunk(tokens, self.settings.price_batch_size)),
        )
    )
    # all rates in EHT: token => rate
    rates_in_eth = reduce(lambda a, b: a | b, batches)
    eth_rate, usd_rate = rates_in_eth[self.settings.native_token_symbol], rates_in_eth[self.settings.stable_token_addr]
    # this gives us the price of 1 eth in usd with 18 decimals precision
    eth_usd_price = (eth_rate * 10 ** eth_decimals) // usd_rate
    # finally convert to prices in terms of stable
    return [Price(token=t, price=(rates_in_eth[t.token_address] * eth_usd_price // 10 ** eth_decimals) / 10 ** eth_decimals) for t in tokens]

### Sign and send transaction

In [ ]:
#| export

@patch
@require_context
async def sign_and_send_tx(self: Chain, tx, value: int = 0, wait: bool = True):
    print(f"sign_and_send_tx: {tx} with value: {value}")
    spender = self.account.address
    tx = await tx.build_transaction({ 'from': spender, 'value': value, 'nonce': await self.web3.eth.get_transaction_count(spender) })
    signed_tx = self.account.sign_transaction(tx)
    tx_hash = await self.web3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return await self.web3.eth.wait_for_transaction_receipt(tx_hash) if wait else tx_hash

### Set and check token allowance

In [ ]:
#| export

@patch
@require_context
async def set_token_allowance(self: Chain, token: Token, addr: str, amount: int):
    ERC20_ABI = [{
        "name": "approve",
        "type": "function",
        "constant": False,
        "inputs": [{"name": "spender", "type": "address"}, {"name": "amount", "type": "uint256"}],
        "outputs": [{"name": "", "type": "bool"}]
    }]
    token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
    return await self.sign_and_send_tx(token_contract.functions.approve(addr, amount))

@patch
@require_context
async def check_token_allowance(self: Chain, token: Token, addr: str) -> int:
    ERC20_ABI = [{
        "name": "allowance",
        "type": "function",
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}, {"name": "spender", "type": "address"}],
        "outputs": [{"name": "", "type": "uint256"}]
    }]
    token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
    return await token_contract.functions.allowance(self.account.address, addr).call()


### Deposit

In [ ]:
#| export

@patch
@require_context
async def deposit(self: Chain, deposit: Deposit, delay_in_minutes: float = 30, slippage: float = 0.01):
    amount_token0, pool, router_contract_addr = deposit.amount_token0, deposit.pool, self.settings.router_contract_addr
    print(f"gonna deposit {amount_token0} {pool.token0.symbol} into {pool.symbol} from {self.account.address}")
    [token0_amount, token1_amount, _] = await self.router.functions.quoteAddLiquidity(
        pool.token0.token_address,
        pool.token1.token_address,
        pool.is_stable,
        pool.factory,
        float_to_uint256(amount_token0, pool.token0.decimals),
        MAX_UINT256
    ).call()
    print(f"Quote: {pool.token0.symbol} {token0_amount / 10 ** pool.token0.decimals} -> {pool.token1.symbol} {token1_amount / 10 ** pool.token1.decimals}")

    # set up allowance for both tokens
    print(f"setting up allowance for {pool.token0.symbol}")
    await self.set_token_allowance(pool.token0, router_contract_addr, token0_amount)

    print(f"setting up allowance for {pool.token1.symbol}")
    await self.set_token_allowance(pool.token1, router_contract_addr, token1_amount)

    # check allowances
    token0_allowance = await self.check_token_allowance(pool.token0, router_contract_addr)
    token1_allowance = await self.check_token_allowance(pool.token1, router_contract_addr)

    print(f"allowances: {token0_allowance}, {token1_allowance}")

    # adding liquidity

    # if token 0 is native, use addLiquidityETH instead of standard addLiquidity
    if pool.token0.token_address == self.settings.wrapped_native_token_addr:
        params = [
            pool.token1.token_address,
            pool.is_stable,
            token1_amount,
            apply_slippage(token1_amount, slippage),
            apply_slippage(token0_amount, slippage),
            self.account.address,
            get_future_timestamp(delay_in_minutes)
        ]
        print(f"adding liquidity with params: {params}")
        return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token0_amount)
    
    # token 1 is native, use addLiquidityETH instead of standard addLiquidity
    if pool.token1.token_address == self.settings.wrapped_native_token_addr:
        params = [
            pool.token0.token_address,
            pool.is_stable,
            token0_amount,
            apply_slippage(token0_amount, slippage),
            apply_slippage(token1_amount, slippage),
            self.account.address,
            get_future_timestamp(delay_in_minutes)
        ]
        print(f"adding liquidity with params: {params}")
        return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token1_amount)

    params = [
        pool.token0.token_address,
        pool.token1.token_address,
        pool.is_stable,
        token0_amount,
        token1_amount,
        apply_slippage(token0_amount, slippage),
        apply_slippage(token1_amount, slippage),
        self.account.address,
        get_future_timestamp(delay_in_minutes)
    ]

    print(f"adding liquidity with params: {params}")

    return await self.sign_and_send_tx(self.router.functions.addLiquidity(*params))

## OP Chain

In [ ]:
#| export

class OPChain(Chain):
    usdc: str = normalize_address("0x7f5c764cbc14f9669b88837ca1490cca17c31607")
    velo: str = normalize_address("0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db")

    def __init__(self, **kwargs): super().__init__(make_op_chain_settings(**kwargs))


## Base Chain

In [ ]:
#| export

class BaseChain(Chain):
    usdc: str = normalize_address("0x833589fcd6edb6e08f4c7c32d4f71b54bda02913")
    aero: str = normalize_address("0x940181a94a35a4569e4529a3cdfb74e38fd98631")

    def __init__(self, **kwargs): super().__init__(make_base_chain_settings(**kwargs))

## Simnet versions of chains

Simnet URIs:

- OP: http://127.0.0.1:4444
- Base: http://127.0.0.1:4445

This assumes the following setup:

```
supersim fork  --l2.host=0.0.0.0 --l2.starting.port=4444 --chains=op,base
```

In [ ]:
#| export 

class OPChainSimnet(OPChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4444", **kwargs)

class BaseChainSimnet(BaseChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4445", **kwargs)

## Tests

Run tests using mainnets for reads and simnet for writes

Make sure simnet is running

In [ ]:
from fastcore.test import test_eq
import socket

host, port ='127.0.0.1', 4444
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex((host, port))
    test_eq(result, 0)
except socket.error as err:
    test_eq(err, None)
finally:
    sock.close()

Make sure PK is set for writes

In [ ]:
import os
from fastcore.test import test_ne
from dotenv import load_dotenv

load_dotenv()

test_ne(os.getenv("SUGAR_PK"), None)

Getting tokens. Make sure native token is included

In [ ]:
from fastcore.test import test_eq, test_ne

async with OPChain() as op:
    tokens = await op.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

async with BaseChain() as base:
    tokens = await base.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

Check on tokens and pricing

In [ ]:
async with OPChain() as chain:
    tokens = await chain.get_all_tokens()
    prices = await chain.get_prices(tokens)
    for p in prices: print(f"{p.token.symbol} price: ${p.price}")

Unclosed client session
client_session: <aiohttp.client.ClientSession object>


Rates in ETH: {'ETH': 1000000000000000000, '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db': 26145154438920, '0x3417E54A51924C225330f8770514aD5560B9098D': 47104965125198, '0x7F5c764cBc14f9669B88837ca1490cCa17c31607': 367417282664231, '0x4200000000000000000000000000000000000006': 1000000000000000000, '0x3E29D3A9316dAB217754d13b28646B76607c5f04': 948741602371318142, '0x6806411765Af15Bddd26f8f544A34cC40cb9838B': 999352861575189755, '0x1F32b1c2345538c0c6f582fCB022739c4A194Ebb': 1193889164138337203, '0xFdb794692724153d1488CcdBE0C56c252596735F': 579290742632677, '0xc40F949F8a4e094D1b49a23ea9241D289B7b2819': 376286070325255, '0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1': 378475307743393, '0xc5b001DC33727F8F26880B184090D3E252470D45': 360609469825393, '0xdFA46478F9e5EA86d57387849598dbFB2e964b02': 63061948318849, '0x4200000000000000000000000000000000000042': 401962558989887, '0x9485aca5bbBE1667AD97c7fE7C4531a624C8b1ED': 392265605273957, '0xCB8FA9a76b8e203D8C3797bF438d8FB81Ea3326A': 370654972344112, '

Basic deposit on OP

In [ ]:
async with OPChain() as chain:
    pools = await chain.get_pools()
    pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo, pools))
    async with OPChainSimnet() as supersim:
        # 0.02 USDC 
        await supersim.deposit(Deposit(pools[0], 0.02))

FOX (0xF1a0DA3367BC7aa04F8D94BA57B862ff37CeD174 None) => 9972926419632
OPP (0x676f784d19c7F1Ac6C6BeaeaaC78B02a73427852 None) => 117541856833
wOptiDoge (0xC26921B5b9ee80773774d36C84328ccb22c3a819 None) => 173841950
fBOMB (0x74ccbe53F77b08632ce0CB91D3A545bF6B8E0979 None) => 11710178780345
JARVIS (0x9F46A936BE9114bD2f71965a87C003341448b47E None) => 27693427774641
OpenX (0xc3864f98f2a61A7cAeb95b039D031b4E2f55e0e9 None) => 11830335121624
MET (0x9a2e53158e12BC09270Af10C16A466cb2b5D7836 None) => 91950774497117
IB (0x00a35FD824c717879BF370E70AC6868b95870Dfb None) => 137704087325102
FXS (0x67CCEA5bb16181E7b4109c9c2143c24a1c2205Be None) => 578856579782223
opxVELO (0xc38464250F51123078BBd7eA574E185F6623d037 None) => 22529360684626
POOL (0x395Ae52bB17aef68C2888d941736A71dC6d4e125 None) => 219981291935972
WBTC (0x68f180fcCe6836688e9084f035309E29Bf0A2095 None) => 3.5642158640753885e+19
tBTC (0x6c84a8f1c29108F47a79964b5Fe888D4f4D0dE40 None) => 35603511262493633979
EXTRA (0x2dAD3a13ef0C6366220f9891570

Web3RPCError: {'code': -32000, 'message': 'out of gas'}

Native -> non Native on OP

In [ ]:
async with OPChain() as op:
    pools = await op.get_pools()
    # find WETH/Velo pool
    weth_velo_pool = next(filter(lambda p: p.token0.token_address == op.settings.wrapped_native_token_addr and p.token1.token_address == OPChain.velo, pools), None)
    test_ne(weth_velo_pool, None)
    async with OPChainSimnet() as supersim:
        await supersim.deposit(Deposit(weth_velo_pool, 0.0001))


gonna deposit 0.0001 WETH into vAMM-WETH/VELO from 0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA
Quote: WETH 0.0001 -> VELO 3.8038150740000978
setting up allowance for WETH
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xa062aE8A9c5e11aaA026fc2670B0D65cCc8B2858', 100000000000000)> with value: 0
setting up allowance for VELO
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xa062aE8A9c5e11aaA026fc2670B0D65cCc8B2858', 3803815074000097664)> with value: 0
allowances: 100000000000000, 3803815074000097664
adding liquidity with params: ['0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', False, 3803815074000097664, 3765776923260097024, 99000000000000, '0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA', 1740396886]
sign_and_send_tx: <Function addLiquidityETH(address,bool,uint256,uint256,uint256,address,uint256) bound to ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', False, 3803815074000097664, 3765776923260097024, 99000000000000, '0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA', 

Basic deposit on Base

In [ ]:
async with BaseChain() as chain:
    pools = await chain.get_pools()
    pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.aero, pools))
    async with BaseChainSimnet() as supersim:
        # 0.02 USDC 
        await supersim.deposit(Deposit(pools[0], 0.01), slippage=0.05)

gonna deposit 0.01 USDC into vAMM-USDC/AERO from 0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA
Quote: USDC 0.01 -> AERO 0.01331474979106487
setting up allowance for USDC
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xcF77a3Ba9A5CA399B7c97c74d54e5b1Beb874E43', 10000)> with value: 0
setting up allowance for AERO
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xcF77a3Ba9A5CA399B7c97c74d54e5b1Beb874E43', 13314749791064871)> with value: 0
allowances: 10000, 13314749791064871
adding liquidity with params: ['0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', '0x940181a94A35A4569E4529A3CDfB74e38FD98631', False, 10000, 13314749791064871, 9500, 12649012301511628, '0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA', 1740396921]
sign_and_send_tx: <Function addLiquidity(address,address,bool,uint256,uint256,uint256,uint256,address,uint256) bound to ('0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', '0x940181a94A35A4569E4529A3CDfB74e38FD98631', False, 10000, 13314749791064871, 9500, 1

CL on OP

In [ ]:
# WIP
# async with OPChain() as chain:
#     pools = await chain.get_pools()
#     # CL200-USDC/VELO
#     cl_usdc_velo = next(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo and x.is_cl, pools), None)
#     test_ne(cl_usdc_velo, None)
#     async with OPChainSimnet() as supersim:
#         await supersim.deposit(Deposit(cl_usdc_velo, 0.02))

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()